# CI / CD with CML API V2

##### Cloudera Machine Learning exposes a REST API that you can use to perform operations related to projects, jobs, and runs. You can use API commands to integrate CML with third-party workflow tools or to control CML from the command line.

##### This notebook demonstrates how to create and execute three CML jobs with the CML API V2. 

For the Public Documentation please visit this [page](https://docs.cloudera.com/machine-learning/cloud/api/topics/ml-api-v2.html)

For a full example of the API's capabilities, please visit this notebook from [Cloudera's Public GitHub](https://github.com/cloudera/CML_AMP_APIv2)

#### To use the Python API in your own code, first install the Python API client and point it to your cluster.

In [5]:
### Install the API

import os
cluster = os.getenv("CDSW_DOMAIN")

In [6]:
from cmlapi.utils import Cursor
import cmlapi
import string
import random
import json

try:
    client = cmlapi.default_client()
except ValueError:
    print("Could not create a client. If this code is not being run in a CML session, please include the keyword arguments \"url\" and \"cml_api_key\".")

session_id = "".join([random.choice(string.ascii_lowercase) for _ in range(6)])
session_id

'nnkkke'

##### The API has a lot of cool features. For example, it allows you to view all runtimes associated with the CML Workspace

In [7]:
# cursor also supports search_filter
# cursor = Cursor(client.list_runtimes, 
#                 search_filter = json.dumps({"image_identifier":"jupyter"}))
cursor = Cursor(client.list_runtimes)
runtimes = cursor.items()
for rt in runtimes:
    print(rt.image_identifier)

aakulov1/cml-zeppelin-runtime:1.0.1
docker.io/cvandyke/automl_pycaret:1.7
docker.io/cvandyke/automl_pycaret:1.8
docker.io/cvandyke/enhancedjupyterlab:1.0
docker.io/cvandyke/enhancedjupyterlab:1.1
docker.io/cvandyke/enhancedjupyterlab:1.2
docker.io/cvandyke/python3.7-java11:1.1
docker.io/cvandyke/python3.7-java11:1.2
docker.io/cvandyke/rstudio:2022.04.9
docker.io/cvandyke/rstudio-cloudera-runtime:2022.04.10
docker.io/cvandyke/rstudio-cloudera-runtime-cpv:2022.12.1
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-cuda:2021.12.1-b17
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-cuda:2022.04.1-b6
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-cuda:2022.11.1-b2
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-rapids:2021.12.1-b17
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-standard:2021.12.1-b17
docker.repository.cloudera.com/cloudera/cdsw/ml-

In [8]:
from __future__ import print_function
import time
import cmlapi
from cmlapi.rest import ApiException
from pprint import pprint

try:
    # List the available runtime addons, optionally filtered, sorted, and paginated.
    api_response = client.list_runtime_addons(page_size=500)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling CMLServiceApi->list_runtime_addons: %s\n" % e)


{'next_page_token': '',
 'runtime_addons': [{'component': 'HadoopCLI',
                     'created_at': datetime.datetime(2022, 5, 26, 15, 28, 57, 53302, tzinfo=tzlocal()),
                     'display_name': 'Hadoop CLI - CDP 7.2.10 - HOTFIX-1',
                     'id': 4,
                     'identifier': 'hadoop-cli-7.2.10-hf1',
                     'manageable': False,
                     'reason': 'PodCompleted: ',
                     'status': 'DELETED'},
                    {'component': 'HadoopCLI',
                     'created_at': datetime.datetime(2022, 10, 21, 19, 4, 43, 251587, tzinfo=tzlocal()),
                     'display_name': 'Hadoop CLI - CDP 7.2.10 - HOTFIX-1 JAVA '
                                     '8U342',
                     'id': 10,
                     'identifier': 'hadoop-cli-7.2.10-hf1-j8u342',
                     'manageable': False,
                     'reason': 'PodCompleted: ',
                     'status': 'AVAILABLE'},
              

##### Similarly, you can see all jobs in the current CML Project

In [9]:
### GET ALL PREVIOUS JOBS FROM PROJECT ###
    
project_id = os.environ["CDSW_PROJECT_ID"]

joblists = client.list_jobs(project_id = project_id)
print(f'Fetched {len(joblists.jobs)} jobs from the project')

Fetched 0 jobs from the project


In [10]:
project_id

'f96y-7e2i-xb6z-6suc'

## You can build ML Ops Pipelines by creating and executing CML Jobs with the API from this notebook.

##### First we create a CML Job to train a model. We trained this model in ProtoType.py but in a real world scenario you may have created your model baseline with ML Flow, also available in CML under the Experiments tab.

In [11]:
!pwd

/home/cdsw/02_APIv2_Pipeline


In [12]:
### CREATE A JOB TO RETRAIN THE MODEL ###
    
# Create a job. We will create dependent/children jobs of this job, so we call this one a "grandparent job". The parameter "runtime_identifier" is needed if this is running in a runtimes project.
monitoring_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "MonitoringJob",
    script = "02_APIv2_Pipeline/01_Pipeline_MonitoringJob.py",
    cpu = 4.0,
    memory = 12.0,
    runtime_identifier = "docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-workbench-python3.7-standard:2022.11.1-b2", 
    runtime_addon_identifiers = ["spark311-13-hf1"]
)

In [13]:
# Create this job within the project specified by the project_id parameter.
monitoring_job = client.create_job(monitoring_job_body, project_id)

##### A second CML Job to push the model to a REST Endpoint can be set to execute as a dependency on the first one

In [14]:
### CREATE A JOB TO PUSH THE MODEL TO A REST ENDPOINT ###

# Create a dependent job by specifying the parent job's ID in the parent_job_id field.
training_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "TrainingJob",
    script = "02_APIv2_Pipeline/02_Pipeline_TrainModelJob.py",
    kernel = "python3",
    cpu = 2,
    memory = 4,
    runtime_identifier = "docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-workbench-python3.7-standard:2022.11.1-b2",
    runtime_addon_identifiers = ["spark311-13-hf1"]
)
training_job = client.create_job(training_job_body, project_id)

##### A third CML Job to push the model to a REST Endpoint can be set to execute as a dependency on Model Training

In [15]:
### CREATE A JOB TO DO INFERENCE ON THE MODEL ###

# Create a job that is dependent on the job from the previous cell. This leads to a dependency chain of grandparent_job -> parent_job -> child_job. If grantparent_job runs and succeeds, then parent_job will trigger, and if parent_job runs and succeeds, child_job will trigger. This one uses a template script that does not terminate, so we'll have the opportunity to try stopping it later.
deploymodel_job_body = cmlapi.CreateJobRequest(
    project_id = project_id,
    name = "DeployModelJob",
    script = "02_APIv2_Pipeline/03_Pipeline_DeployModelJob.py",
    kernel = "python3",
    cpu = 4,
    memory = 12,  
    runtime_identifier = "docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-workbench-python3.7-standard:2022.11.1-b2",
    runtime_addon_identifiers = ["spark311-13-hf1"],
    parent_job_id = training_job.id
)
deploymodel_job = client.create_job(deploymodel_job_body, project_id)

##### Notice that although we have created the jobs, we haven't executed them yet

##### If you hover over to the CML Project landing page and open the Jobs tab you will notice the jobs have been added under the "Jobs Section"

![title](images/cml-jobs-created.png)

##### Next, we can use the API to run the first job. When it succeeds, the second job will execute, and then the third...

In [16]:
# Create a job run for the specified job.
# If the job has dependent jobs, the dependent jobs will run after the job succeeds.
# In this case, the grandparent job will run first, then the parent job, and then the child job, provided each job run succeeds.
jobrun_body = cmlapi.CreateJobRunRequest(project_id, monitoring_job.id)
job_run = client.create_job_run(jobrun_body, project_id, monitoring_job.id)
run_id = job_run.id

##### Go back to the Jobs tab in the CML landing page. Notice the jobs are now executing 

![title](images/cml-jobs-running.png)

##### If you want to learn more about CML Jobs, you can click on the CML Job and four new tabs will open, giving you the ability to explore the Job DAG, look into execution, and even determine where in your code errors occurred if any

![title](images/cml-job-dependencies.png)

![title](images/cml-job-history.png)

![title](images/cml-job-troubleshoot.png)